# Query/Context Dataset Generation
***

This notebook walks students through the process of generating datasets of query/context pairs which can be used for two primary purposes:
- Fine-tuning an embedding model
- Serve as ground truth for retrieval evaluation

In [1]:
%load_ext autoreload 
%autoreload 2

import sys
sys.path.append('../')

# from src.evaluation.retrieval_evaluation import QueryContextGenerator
# from llama_index.finetuning import EmbeddingQAFinetuneDataset
from src.llm.prompt_templates import qa_generation_prompt
from src.llm.llm_interface import LLM
from src.preprocessor.preprocessing import FileIO
from rich import print
import random
import pandas as pd
import os

from dotenv import load_dotenv
env = load_dotenv('.env', override=True)

In [2]:
print(qa_generation_prompt)

Huberman Lab episode summary and episode guest are below:

---------------------
Summary: {summary}
---------------------
Guest: {guest}
---------------------
Given the Summary and Guest of the episode as context use the following randomly selected transcript section of the
episode and not prior knowledge, generate questions that can be answered by the transcript section: 

---------------------
Transcript: {transcript}
---------------------

Your task is to create {num_questions_per_chunk} questions that can only be answered given the previous context and
transcript details and no other information. The question should randomly start with How, Why, or What.

In [7]:
llm = LLM(model_name='azure/gpt-35-turbo', 
          api_key=os.environ['AZURE_OPENAI_API_KEY'],
          api_version=os.environ['AZURE_OPENAI_API_VERSION'],
          api_base=os.environ['AZURE_OPENAI_ENDPOINT'])

In [8]:
generator = QueryContextGenerator(llm)

In [16]:
data_path = '../data/huberman_minilm-512.parquet'
data = FileIO().load_parquet(data_path)

Shape of data: (11602, 13)
Memory Usage: 1.15+ MB


In [5]:
class QueryContextGenerator:
    '''
    Class designed for the generation of query/context pairs using a
    Generative LLM. The LLM is used to generate questions from a given
    corpus of text. The query/context pairs can be used to fine-tune 
    an embedding model using a MultipleNegativesRankingLoss loss function
    or can be used to create evaluation datasets for retrieval models.
    '''
    def __init__(self, 
                 llm: LLM
                ):
        self.llm = llm

    def clean_validate_data(self,
                            data: list[dict], 
                            valid_fields: list[str]=['content', 'summary', 'guest', 'doc_id'],
                            total_chars: int=950
                            ) -> list[dict]:
        '''
        Strip original data chunks so they only contain valid_fields.
        Remove any chunks less than total_chars in size. Prevents LLM
        from asking questions from sparse content. 
        '''
        clean_docs = [{k:v for k,v in d.items() if k in valid_fields} for d in data]
        valid_docs = [d for d in clean_docs if len(d['content']) > total_chars]
        return valid_docs

    def train_val_split(self,
                        data: list[dict],
                        n_train_questions: int, 
                        n_val_questions: int, 
                        n_questions_per_chunk: int=2,
                        total_chars: int=950):
        '''
        Splits corpus into training and validation sets.  Training and 
        validation samples are randomly selected from the corpus. total_chars
        parameter is set based on pre-analysis of average doc length in the 
        training corpus. 
        '''
        clean_data = self.clean_validate_data(data, total_chars=total_chars)
        random.shuffle(clean_data)
        train_index = n_train_questions//n_questions_per_chunk
        valid_index = n_val_questions//n_questions_per_chunk
        end_index = valid_index + train_index
        if end_index > len(clean_data):
            raise ValueError('Cannot create dataset with desired number of questions, try using a larger dataset')
        train_data = clean_data[:train_index]
        valid_data = clean_data[train_index:end_index]
        print(f'Length Training Data: {len(train_data)}')
        print(f'Length Validation Data: {len(valid_data)}')
        return train_data, valid_data

    def generate_qa_embedding_pairs(
                                    self,
                                    data: list[dict],
                                    generate_prompt_tmpl: str,
                                    system_message: str='You are a helpful assistant.',
                                    num_questions_per_chunk: int = 2,
                                    ) -> dict:
        """
        Generate query/context pairs from a list of documents. The query/context pairs
        can be used for fine-tuning an embedding model using a MultipleNegativesRankingLoss
        or can be used to create an evaluation dataset for retrieval models.
        """
        queries = {}
        relevant_docs = {}
        corpus = {chunk['doc_id'] : chunk['content'] for chunk in data}
        for chunk in tqdm(data):
            summary = chunk['summary']
            guest = chunk['guest']
            transcript = chunk['content']
            doc_id = chunk['doc_id']
            assist_message = generate_prompt_tmpl.format(summary=summary, 
                                                         guest=guest,
                                                         transcript=transcript,
                                                         num_questions_per_chunk=num_questions_per_chunk)
            try:
                response = self.llm.chat_completion(system_message, 
                                                    assist_message, 
                                                    temperature=1.0, 
                                                    max_tokens=num_questions_per_chunk*50,
                                                    return_raw=False
                                                   )
            except Exception as e:
                print(e)
                continue
            result = response.strip().split("\n")
            questions = [
                re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
            ]
            questions = [question for question in questions if len(question) > 0]

            for question in questions:
                question_id = str(uuid.uuid4())
                queries[question_id] = question
                relevant_docs[question_id] = [doc_id]

        # construct dataset
        return dict(queries=queries, corpus=corpus, relevant_docs=relevant_docs)

# def execute_evaluation(dataset: dict, 
#                        class_name: str, 
#                        retriever: str,
#                        reranker: ReRanker=None,
#                        alpha: float=0.5,
#                        retrieve_limit: int=100,
#                        top_k: int=5,
#                        chunk_size: int=256,
#                        hnsw_config_keys: list[str]=['maxConnections', 'efConstruction', 'ef'],
#                        search_type: Literal['hybrid', 'all']='all',
#                        search_properties: list[str]=['content'],
#                        display_properties: list[str]=['doc_id', 'content'],
#                        dir_outpath: str='./eval_results',
#                        include_miss_info: bool=False,
#                        user_def_params: dict=None
#                        ) -> Union[dict, Tuple[dict, list[dict]]]:
#     '''
#     Given a dataset, a retriever, and a reranker, evaluate the performance of the retriever and reranker. 
#     Returns a dict of kw, vector, and hybrid hit rates and mrr scores. If inlude_miss_info is True, will
#     also return a list of kw and vector responses and their associated queries that did not return a hit.

#     Args:
#     -----
#     dataset: EmbeddingQAFinetuneDataset
#         Dataset to be used for evaluation
#     class_name: str
#         Name of Class on Weaviate host to be used for retrieval
#     retriever: WeaviateClient
#         WeaviateClient object to be used for retrieval 
#     reranker: ReRanker
#         ReRanker model to be used for results reranking
#     alpha: float=0.5
#         Weighting factor for BM25 and Vector search.
#         alpha can be any number from 0 to 1, defaulting to 0.5:
#             alpha = 0 executes a pure keyword search method (BM25)
#             alpha = 0.5 weighs the BM25 and vector methods evenly
#             alpha = 1 executes a pure vector search method
#     retrieve_limit: int=5
#         Number of documents to retrieve from Weaviate host
#     top_k: int=5
#         Number of top results to evaluate
#     chunk_size: int=256
#         Number of tokens used to chunk text
#     hnsw_config_keys: list[str]=['maxConnections', 'efConstruction', 'ef']
#         list of keys to be used for retrieving HNSW Index parameters from Weaviate host
#     search_type: Literal['kw', 'vector', 'hybrid', 'all']='all'
#         Type of search to be evaluated.  Options are 'kw', 'vector', 'hybrid', or 'all'
#     search_properties: list[str]=['content']
#         list of properties to be used for search
#     display_properties: list[str]=['doc_id', 'content']
#         list of properties to be returned from Weaviate host for display in response
#     dir_outpath: str='./eval_results'
#         Directory path for saving results.  Directory will be created if it does not
#         already exist. 
#     include_miss_info: bool=False
#         Option to include queries and their associated search response values
#         for queries that are "total misses"
#     user_def_params : dict=None
#         Option for user to pass in a dictionary of user-defined parameters and their values.
#         Will be automatically added to the results_dict if correct type is passed.
#     '''
        
#     reranker_name = reranker.model_name if reranker else "None"
    
#     results_dict = {'n':retrieve_limit, 
#                     'top_k': top_k,
#                     'alpha': alpha,
#                     'Retriever': retriever.model_name_or_path, 
#                     'Ranker': reranker_name,
#                     'chunk_size': chunk_size,
#                     'hybrid_hit_rate':0,
#                     'hybrid_mrr': 0,
#                     }
#     #add extra params to results_dict
#     kw_vector_metrics = {
#                          'kw_hit_rate': 0,
#                          'kw_mrr': 0,
#                          'vector_hit_rate': 0,
#                          'vector_mrr': 0
#                          }
#     if search_type == 'all':
#         results_dict = {**results_dict, **kw_vector_metrics}
#     results_dict['total_questions'] = 0
#     results_dict['total_misses'] = 0
#     results_dict = add_params(retriever, class_name, results_dict, user_def_params, hnsw_config_keys)
        
#     start = time.perf_counter()
#     miss_info_list = []
#     for query_id, q in tqdm(dataset.queries.items(), 'Queries'):
#         results_dict['total_questions'] += 1
#         hit = False
#         #make Keyword, Vector, and Hybrid calls to Weaviate host
#         try:
#             hybrid_response = retriever.hybrid_search(request=q, class_name=class_name, properties=search_properties, alpha=alpha, limit=retrieve_limit, display_properties=display_properties)  
#             if search_type == 'all':
#                 kw_response = retriever.keyword_search(request=q, class_name=class_name, properties=search_properties, limit=retrieve_limit, display_properties=display_properties)
#                 vector_response = retriever.vector_search(request=q, class_name=class_name, limit=retrieve_limit, display_properties=display_properties)         
#             #rerank returned responses if reranker is provided
#             if reranker:
#                 hybrid_response = reranker.rerank(hybrid_response, q, top_k=top_k)
#                 if search_type == 'all':
#                     kw_response = reranker.rerank(kw_response, q, top_k=top_k)
#                     vector_response = reranker.rerank(vector_response, q, top_k=top_k)
                
            
#             #collect doc_ids to check for document matches (include only results_top_k)
#             hybrid_doc_ids = {result['doc_id']:i for i, result in enumerate(hybrid_response[:top_k], 1)}
#             if search_type == 'all':
#                 kw_doc_ids = {result['doc_id']:i for i, result in enumerate(kw_response[:top_k], 1)}
#                 vector_doc_ids = {result['doc_id']:i for i, result in enumerate(vector_response[:top_k], 1)}
#             #extract doc_id for scoring purposes
#             doc_id = dataset.relevant_docs[query_id][0]
     
#             #increment hit_rate counters and mrr scores
#             if doc_id in hybrid_doc_ids:
#                 results_dict['hybrid_hit_rate'] += 1
#                 results_dict['hybrid_mrr'] += 1/hybrid_doc_ids[doc_id]
#                 hit = True
#             if search_type == 'all':
#                 if doc_id in kw_doc_ids:
#                     results_dict['kw_hit_rate'] += 1
#                     results_dict['kw_mrr'] += 1/kw_doc_ids[doc_id]
#                     hit = True
#                 if doc_id in vector_doc_ids:
#                     results_dict['vector_hit_rate'] += 1
#                     results_dict['vector_mrr'] += 1/vector_doc_ids[doc_id]
#                     hit = True
#             # if no hits, let's capture that
#             if not hit:
#                 results_dict['total_misses'] += 1
#                 miss_info = {'query': q, 
#                              'answer': dataset.corpus[doc_id],
#                              'doc_id': doc_id,
#                              'hybrid_response': hybrid_response}
#                 if search_type == 'all':
#                     miss_info = {**miss_info, 
#                                  'kw_response': kw_response,
#                                  'vector_response': vector_response}
#                 miss_info_list.append(miss_info)

#         except Exception as e:
#             print(e)
#             continue

#     #use raw counts to calculate final scores
#     calc_hit_rate_scores(results_dict, search_type=search_type)
#     calc_mrr_scores(results_dict, search_type=search_type)
    
#     end = time.perf_counter() - start
#     print(f'Total Processing Time: {round(end/60, 2)} minutes')
#     record_results(results_dict, chunk_size, dir_outpath=dir_outpath, as_text=True)
    
#     if include_miss_info:
#         return results_dict, miss_info
#     return results_dict

# def calc_hit_rate_scores(results_dict: Dict[str, Union[str, int]], 
#                          search_type: Literal['hybrid', 'all']='all'
#                          ) -> None:
#     '''
#     Helper function to calculate hit rate scores
#     '''
#     search_type = ['kw', 'vector', 'hybrid'] if search_type == 'all' else [search_type]
#     for prefix in search_type:
#         results_dict[f'{prefix}_hit_rate'] = round(results_dict[f'{prefix}_hit_rate']/results_dict['total_questions'],2)

# def calc_mrr_scores(results_dict: Dict[str, Union[str, int]],
#                     search_type: Literal['hybrid', 'all']='all'
#                     ) -> None:
#     '''
#     Helper function to calculate mrr scores
#     '''
#     search_type = ['kw', 'vector', 'hybrid'] if search_type == 'all' else [search_type]
#     for prefix in search_type:
#         results_dict[f'{prefix}_mrr'] = round(results_dict[f'{prefix}_mrr']/results_dict['total_questions'],2)

# def create_dir(dir_path: str) -> None:
#     '''
#     Checks if directory exists, and creates new directory
#     if it does not exist
#     '''
#     if not os.path.exists(dir_path):
#         os.makedirs(dir_path)

# def record_results(results_dict: Dict[str, Union[str, int]], 
#                    chunk_size: int, 
#                    dir_outpath: str='./eval_results',
#                    as_text: bool=False
#                    ) -> None:
#     '''
#     Write results to output file in either txt or json format

#     Args:
#     -----
#     results_dict: Dict[str, Union[str, int]]
#         Dictionary containing results of evaluation
#     chunk_size: int
#         Size of text chunks in tokens
#     dir_outpath: str
#         Path to output directory.  Directory only, filename is hardcoded
#         as part of this function.
#     as_text: bool
#         If True, write results as text file.  If False, write as json file.
#     '''
#     create_dir(dir_outpath)
#     time_marker = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
#     ext = 'txt' if as_text else 'json'
#     path = os.path.join(dir_outpath, f'retrieval_eval_{chunk_size}_{time_marker}.{ext}')
#     if as_text:
#         with open(path, 'a') as f:
#             f.write(f"{results_dict}\n")
#     else: 
#         with open(path, 'w') as f:
#             json.dump(results_dict, f, indent=4)

# def add_params(client: WeaviateClient, 
#                class_name: str, 
#                results_dict: dict, 
#                param_options: dict, 
#                hnsw_config_keys: list[str]
#               ) -> dict:
#     '''
#     Helper function that adds parameters to the results_dict:
#     - Adds HNSW Index parameters to results_dict
#     - Adds optional user-defined parameters to results_dict
#     '''
#     hnsw_params = {k:v for k,v in client.show_class_config(class_name)['vectorIndexConfig'].items() if k in hnsw_config_keys}
#     if hnsw_params:
#         results_dict = {**results_dict, **hnsw_params}
#     if param_options and isinstance(param_options, dict):
#         results_dict = {**results_dict, **param_options}
#     return results_dict

### Load raw data
Load raw data from parquet file.  Raw data should be in the same format as the dataset (corpus) created in [Notebook 1](https://github.com/americanthinker/vectorsearch-applications/blob/main/1-Data_Preprocessing_Week1_COLAB.ipynb). 

In [4]:
data_path = './data/impact_theory_minilm_256.parquet'
data = FileIO().load_parquet(data_path)
len(data)

Shape of data: (26448, 12)
Memory Usage: 2.42+ MB


26448

### Data Length Analysis
Conduct an analysis of the length of the content chunks.  Can use either raw words or tokens to assess length.  The main point here is to get a sense of the mean length of content chunks in the data and to set the `total_chars` param in the `clean_validate_data` method with an appropriate value.

In [5]:
#in this example the mean content length is @ 1,000
lengths = [len(d['content']) for d in data]
df = pd.DataFrame(lengths)
df.describe() 

,0
count,26448.000000
mean,991.729053
std,126.344870
min,4.000000
25%,944.000000
50%,1005.000000
75%,1060.000000
max,1974.000000


### Split Data

The `train_val_split` function will clean and validate the raw data as a first step and then split into user defined train/val splits.  
- Cleaning simply strips the keys from the data that are not needed for the query/content generation process
- Validation consists of ensuring that only content chunks of length > `total_chars` are passed to the LLM (this step prevents the LLM from asking questions from sparse context)

Users define the number of training samples and validation samples to generate.  Number of questions per content chunk can also be set to more than 1, however a note of caution:
- Setting `num_questions_per_chunk` > 1 saves time (and money) by asking more than one question per content chunk, however, the dataset will be less diverse.  There is also the potential for the model to generate lower quality questions if the content chunk isn't large enough or meaningful enough to generate more than one question from the content.
- Retrieval evaluation results from fine-tuning an embedding model with 200-300 training samples showed an uptick of 5-10% points.  Upper bound on retrieval improvement as a funtion of training sample size is yet to be determined (have fun pushing the boundaries! 👊)
- A validation data set is not required for seeing improvement from fine tuning.  The addition of a validation dataset, however, allows a user to test the results of fine tuning on an unseen dataset. 

In [6]:
#split data into train/val sets
#in this example we are creating a training set of n=10, val set of n=5, and asking the LLM to only ask 1 question per chunk. 
train, val = generator.train_val_split(data, 10, 5, 1, total_chars=950)

Length Training Data: 10
Length Validation Data: 5


### Generate QA pairs

To generate query/context pairs we need to pass in our cleaned data splits, a question asking generation prompt, and the number of questions per chunk (needs to be same value passed into the `train_val_split` function.
The `qa_generation_prompt` is already preconfigured and supplies the LLM with additional context about the Impact Theory show to ensure high quality questions are asked given the additional context.   
Print out the prompt to see what is being asked of the model:

In [7]:
print(qa_generation_prompt)

Impact Theory episode summary and episode guest are below:

---------------------
Summary: {summary}
---------------------
Guest: {guest}
---------------------
Given the Summary and Guest of the episode as context use the following randomly selected transcript section \  
of the episode and not prior knowledge, generate questions that can be answered by the transcript section: 

---------------------
Transcript: {transcript}
---------------------

Your task is to create {num_questions_per_chunk} questions that can only be answered given the previous context and
transcript details. The question should randomly start with How, Why, or What.

The output from this function is a llama_index class `EmbeddingQAFinetuneDataset`, which is a simple wrapper for a series of three dictionaries (`corpus`, `queries`, and `relevant_docs`).  The llama_index class is not absolutely necessary, but it is helpful in making transitions smoother when using the llama_index `SentenceTransformersFinetuneEngine` class for fine-tuning.  It takes roughly 80 seconds to generate 100 query/context pairs so a sample size of 300 takes about 4 minutes (much faster than if you were to do this manually!).

In [12]:
training_set = generator.generate_qa_embedding_pairs(train, qa_generation_prompt, 2)
# val_set = generator.generate_qa_embedding_pairs(val, qa_generation_prompt, 1)

100%|████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.30s/it]


In [13]:
#EmbeddingQAFinetuneDataset has no len, so check length of queries instead
len(training_set.queries), len(val_set.queries)

(20, 5)

### Dataset Analysis

Always a good idea to check the quality of the pairs generated.  Most pairs will be high quality but some will not be, this is a chance for human intervention to adjust the questions manually to ensure the quality remains high. 

In [14]:
def show_qa_pairs(data: EmbeddingQAFinetuneDataset, print_results: bool=True):
    pairs = []
    for k, v in data.queries.items():
        doc_id = data.relevant_docs[k][0]
        context = data.corpus[doc_id]
        pairs.append((v, context))
    if print_results:
        for tup in pairs:
            print(f'Question: {tup[0]}\nContext: {tup[1]}\n\n')
    return pairs    

In [17]:
pairs = show_qa_pairs(training_set, print_results=True)

Question: How does Peter Attia approach cancer treatment for his patients?
Context: Is it one that responds to immunotherapy? Is there like a protocol that you would walk down? No, it would 
be entirely dependent on what the cancer was. So that gets to kind of step two of my thinking on cancer, which is 
the unequivocal, unambiguous understanding that your odds for treating cancer go down the more cancer cells you 
have in your body. So detecting it sooner. In the book, you talk about you, for your patients, you lower the age of
colonoscopy. We do everything much more aggressively. So we're doing colonoscopy much earlier, much more 
frequently. How often would you do a colonoscopy? Oh, it depends on the individual. I mean, in me, I do it every 
three years. Really? Yeah, and then I do stool-based testing in between. What do you check for in the stool? You're
looking for fecal DNA. So you're looking for DNA of the colon cancer, yeah. Interesting. It has its own DNA? What 
are we looking for? Yeah, you're looking for the DNA that's shedding from a tumor.

Question: What does Peter Attia check for in stool-based testing for colon cancer?
Context: Is it one that responds to immunotherapy? Is there like a protocol that you would walk down? No, it would 
be entirely dependent on what the cancer was. So that gets to kind of step two of my thinking on cancer, which is 
the unequivocal, unambiguous understanding that your odds for treating cancer go down the more cancer cells you 
have in your body. So detecting it sooner. In the book, you talk about you, for your patients, you lower the age of
colonoscopy. We do everything much more aggressively. So we're doing colonoscopy much earlier, much more 
frequently. How often would you do a colonoscopy? Oh, it depends on the individual. I mean, in me, I do it every 
three years. Really? Yeah, and then I do stool-based testing in between. What do you check for in the stool? You're
looking for fecal DNA. So you're looking for DNA of the colon cancer, yeah. Interesting. It has its own DNA? What 
are we looking for? Yeah, you're looking for the DNA that's shedding from a tumor.

Question: How did Jim Kwik's struggles with learning challenges and fear of public speaking ultimately benefit him 
in his career?
Context: Like that's like the master energy control center. So for me, I would always lean into my practices 
because it forced me to double down. I think there's a gift. I don't know if it's true enough, but it's been my 
experience where when people go through struggles that there's a gift in this, like whether there's a gift of 
what's going on right now, true or false, I choose to believe it because then I'll operate from that, from that 
point of view and that perspective, meaning what's the gift in me having learning challenges and fearful of public 
speaking while I got really damn good at learning and public speaking because that's all I do now for, for, for a 
living, right? What was the advantage that came out of this sleep deprivation experiment for 10 years now without 
that much better is I could tell you two things immediately. It forced me to double down on everything I teach 
because I'm just, I'm just documenting and telling people what I do, right? Otherwise I won't be able to perform at
the level that I do. And then number two, it's forced me also to be very selective in the things I say yes to.

Question: What were the two immediate advantages that Jim Kwik gained from his sleep deprivation experiment?
Context: Like that's like the master energy control center. So for me, I would always lean into my practices 
because it forced me to double down. I think there's a gift. I don't know if it's true enough, but it's been my 
experience where when people go through struggles that there's a gift in this, like whether there's a gift of 
what's going on right now, true or false, I choose to believe it because then I'll operate from that, from that 
point of view and that perspective, meaning what's the gift in me having learning challenges and fearful of public 
speaking while I got really damn good at learning and public speaking because that's all I do now for, for, for a 
living, right? What was the advantage that came out of this sleep deprivation experiment for 10 years now without 
that much better is I could tell you two things immediately. It forced me to double down on everything I teach 
because I'm just, I'm just documenting and telling people what I do, right? Otherwise I won't be able to perform at
the level that I do. And then number two, it's forced me also to be very selective in the things I say yes to.

Question: How does Kurt Busch describe the balance between professionalism and tenacity in his racing career?
Context: There's some team owners or some mechanics or team personnel that are with you, but they have more of a 
balanced political approach to it, and then there's some that are just like, you know what? Let's let six months go
by, and then we'll reevaluate. In racing timelines, that's way too long. You got to move quick, and that's what was
a lot of my not balancing the professionalism with my tenacity or talent from behind the wheel. What do you plan to
do in the next phase? At some point, I imagine you will say, okay, I'm going to step away from driving. What do you
want to see the next phase of your life? Will it be equally competitive? Is there something that you already have 
on tap that you want to do? I'm sure I'll be competitive. I married a beautiful woman who is competitive in her own
rights. My wife, Ashley, plays competitive polo, and she has her horses, her team, and travels, and she has the 
same thing that I have. It's called helmetitis. You put a helmet on, and you change into a different person.

...

Question: Why does Evan Puschak believe that writing a lot makes it easier to compose a persuasive story?
Context: You've done such a good job of taking a subject, like Rihanna's work, work, work, work, work song, like 
you've got a whole fucking show about that song. Tom Bilyeu I love that video. I love making that video. Tom Bilyeu
How is this happening? He made a show, he spent a week on this song and it's really interesting. Tom Bilyeu 
Hardcore week. Tom Bilyeu How did you train yourself to go deep like that? I think when you write a lot, it makes 
it a little bit easier to compose a story. That part of my brain and mind is still primed for taking information 
and composing it into something that is persuasive and like a story. Every week or whenever I come up with 
something for the nerd writer, it's usually a combination of some kind of introspective thought process that I want
to talk about and something from the world that I've consumed that I think is interesting and I like to talk about 
it and it's like, oh there's a good interaction between those two, let's see what I can do there.

Question: How did the mice in the study gain weight and body fat despite eating the same amount of calories?
Context: Those researchers at the Wiseman Institute who understand about what's, you know, the bacteria and mice, 
they took bacteria samples, fecal samples, which fecal transplantation is like one of the hottest things on the 
street as well. It's super weird, but it is. But they take they took these fecal samples from folks who had a 
bacteria cascade associated with obesity and implanted it into lean mice. And they took another set of fecal 
samples from human subjects who had a bacteria cascade associated with leanness and implanted that into lean mice. 
Those mice stayed lean. The mice who received the implants from the folks with the bacteria cascade associated with
obesity, those mice became insulin resistant. They gained weight and gained body fat, not because of calories, not 
because they changed what they were eating because of the bacteria. These principles supersede any of the ideas 
that we carry about just managing calories. If you just get into a caloric deficit because the mice are already 
eating the same thing, yet they're gaining weight.

Question: What did the researchers find when they implanted fecal samples from individuals with a bacteria cascade 
associated with obesity into lean mice?
Context: Those researchers at the Wiseman Institute who understand about what's, you know, the bacteria and mice, 
they took bacteria samples, fecal samples, which fecal transplantation is like one of the hottest things on the 
street as well. It's super weird, but it is. But they take they took these fecal samples from folks who had a 
bacteria cascade associated with obesity and implanted it into lean mice. And they took another set of fecal 
samples from human subjects who had a bacteria cascade associated with leanness and implanted that into lean mice. 
Those mice stayed lean. The mice who received the implants from the folks with the bacteria cascade associated with
obesity, those mice became insulin resistant. They gained weight and gained body fat, not because of calories, not 
because they changed what they were eating because of the bacteria. These principles supersede any of the ideas 
that we carry about just managing calories. If you just get into a caloric deficit because the mice are already 
eating the same thing, yet they're gaining weight.

Question: How does Bjorn Lomborg emphasize the importance of prioritizing solutions based on cost-effectiveness and
impact?
Context: We should have good jobs for everyone. And we should have organic apples for everyone and community 
gardens and the whole thing. And you're sort of like, really? Yes, of course. I would love this world where we had 
everything to everyone. But clearly if you're promising everything to everyone, you have no priorities. You're 
literally not giving any direction. You're just saying all good things in apple pie. But why is that bad? So that 
sounds amazing. And I think part of people's hangup is what's the problem? Like that sounds awesome. Yes, let's do 
it. And Bjorn, there are so many people in the world. There's so much wealth. Like, come on, come on. Can't Elon 
Musk solve these problems by himself? So I can put it to you in a way I love, which is just numbers. So if you try 
to cost how much this is going to cost, it'll probably cost an additional 10 to $15 trillion. To give you a sense 
of proportion right now, the global tax intake of all governments in the world is $15 trillion.

Question: What is the estimated additional cost of providing everything to everyone, according to Bjorn Lomborg?
Context: We should have good jobs for everyone. And we should have organic apples for everyone and community 
gardens and the whole thing. And you're sort of like, really? Yes, of course. I would love this world where we had 
everything to everyone. But clearly if you're promising everything to everyone, you have no priorities. You're 
literally not giving any direction. You're just saying all good things in apple pie. But why is that bad? So that 
sounds amazing. And I think part of people's hangup is what's the problem? Like that sounds awesome. Yes, let's do 
it. And Bjorn, there are so many people in the world. There's so much wealth. Like, come on, come on. Can't Elon 
Musk solve these problems by himself? So I can put it to you in a way I love, which is just numbers. So if you try 
to cost how much this is going to cost, it'll probably cost an additional 10 to $15 trillion. To give you a sense 
of proportion right now, the global tax intake of all governments in the world is $15 trillion.

### Save to Disk  
Save to disk using your own filepaths, below is an example using the length of the sets as part of the filepath.

In [28]:
# training_set.save_json('./data/training_data_10.json')
# val_set.save_json('./data/validation_data_5.json')